# Uso de WRF-Python para la extracción de variables:

##  Laboratorio Avanzado 2. II Ciclo 2019
### Manuel Chaves Alvarez.
### Contacto: manuelczh@hotmail.com.ar

La documentación completa de la librería puede ser encontrada en: https://wrf-python.readthedocs.io/en/latest/basic_usage.html



### Se importan los modulos necesarios, mas adelante se detallara su uso, en los casos que su uso no sea obvio.

In [ ]:
from __future__ import print_function
from netCDF4 import Dataset, MFDataset
from collections import OrderedDict
from wrf import getvar, ALL_TIMES, interplevel, CoordPair, xy_to_ll, ll_to_xy, disable_xarray
import numpy as np
from numpy import shape
import xarray as xr
from calendar import monthrange
import time
import concurrent.futures
from pathlib import Path

### Lectura de los paths de archivos de la base de datos CA-CORDEX

Se itera sobre el sistema de archivos de la base de datos esto con el el fin de recuperar la direcciones de cada uno de las salidas del modelo, para este caso los dos años seleccionados 1981 y 2013, ademas se usan los comandos "magicos" %. 

``` python
# condicion que limita a obtener salidas a las 12Z, en caso de no ser necesario 
# eliminar la linea por completo
if "12"==content[i][22:24]
```


Nota: Las siguientes $\textbf{dos celdas}$ pueden ser reemplazadas por un "script de bash" y luego ejecutada de nuevo usando los comandos magicos %.

In [ ]:
año1 = '1981'
año2 = '2013'
meses=['01','02','03','04','05','06','07','08','09','10','11','12']

In [ ]:
## La direccion de CA-CORDEX es la existente para la fecha de realizacion de este documento
## y podra variar de acuerdo al centro.

%cd /storage/nfs/CA-CORDEX/$año2
for i in ['01','02','03','04','05','06','07','08','09','10','11','12']:
    %cd /storage/nfs/CA-CORDEX/$año2/$i
    %ls >> /home/cigefi/$año2
    %cd ..
    
%cd
with open(año2) as f:
    content = f.readlines()

## Se eliminan los cambios de linea
content = [x.strip() for x in content] 
nclist=[]
## Se seleccionan aquellas direcciones cuya salida sea a las 12Z
## [22:24] es donde se encuentra la hora de la salida en el string de content[i]
nclist.append([content[i] for i in np.arange(len(content)) if "12"==content[i][22:24]])    

## fill, almacena las direcciones de todos los archvos
fill = np.zeros(365, dtype=object)   
dataset_list = []
days =[]
for k in np.arange(1,13):
    # la eleccion del año 1982 es indiferente mientras se cumpla que el año NO SEA bisiesto
    
    num_days = monthrange(1982, k)[1] 
    days.append(num_days)

array = np.array([days,meses])    
for i in np.arange(0,365):
    mes = nclist[0][i][16:18]
    %cd /storage/nfs/CA-CORDEX/$año2/$mes
    fill[i]=str(Path(nclist[0][i]).absolute())

## Extraccion de variables del set de datos

#### Por medio de loop and fill (ver documentación en la sección de PERFOMANCE TIPS)
#### Estaciones: Barbados, Guadalupe, San Andres.

Aquí se define la función: 
``` python 
def extrac_var_dynamics(var,station,var2,station2,a,b) 
``` 
Cada uno de sus argumentos (listas) definidos de la forma: 


**var** y **var2**: la variable que se quiera extraer de acuerdo a la documentación de wrf-python, se definen de forma separada (con unidades a escoger **var**, unidades por defecto **var2**), ya que **getvar** tiene argumentos específicos en cuanto a unidades.

**station** y **station2**: nombre de salidas de las variables dadas por var y var2 respectivamente.


**a y b**: puntos de grilla más cercanos a las estaciones.

**NOTA 1:** el *len()* de los argumentos **IMPORTA** todos tienen que tener la misma dimensión
si se deseara cambiar se debe de tomar en cuenta lo anterior.

**NOTA 2:** los archivos de salida están en formato **.npy**

**NOTA 3:** Para las 3 estaciones con sus 6 variables (1 TB de información, interpoladas a 14 niveles de presión) para 1 año, el tiempo aproximado de ejecución es de 2 horas (Usando 4 núcleos). 


In [ ]:
# de acuerdo a donde queira guradar los datos.
%cd
%cd Escritorio/

In [ ]:
from __future__ import print_function, division

In [ ]:
#esto elimina el metadato que se extare en cada variable, ahorra memoria.
disable_xarray()

#niveles de presion deseados para la interpolacion
lista = [1000.0, 925.0, 850.0, 700.0, 500.0, 400.0, 300.0, 250.0, 200.0, 150.0, 100.0, 70.0, 50.0,10.0]

#puntos de grilla mas cercanos
grid = [[84,219],[93,286],[79,213]]
grid_points = np.array(grid +grid)

station=['zonalBARB','zonalGUAD','zonalSAND','meridionalBARB','meridionalGUAD','meridionalSAND']
#variables de veitnp zonal y meridional
var=['ua','ua','ua','va','va','va']

station2=['qBARB','qGUAD','qSAND','temperaturaBARB','temperaturaGUAD','temperaturaSAND']
#variables de temperatura y QVAPOR
var2=['QVAPOR','QVAPOR','QVAPOR','tc','tc','tc']

In [ ]:
def extrac_var_dynamics(var,station,var2,station2,a,b):

    filename_list = fill
    # 14 niveles de presion por 365 dias del año
    result_shape  = (14,365)
    print(fill[0][42:46])
    # se crean arrays vacios con el fin de llenarlos con los datos que se extraen
    var_temp  = np.empty(result_shape, np.float32)
    var_temp2 = np.empty(result_shape, np.float32)
    print(station,var,"........processing")
    print(station2,var2,"........processing")
    times_per_file = 1
    
    for timeidx in range(result_shape[1]):
        
        fileidx = timeidx // times_per_file
        file_timeidx = timeidx % times_per_file
        #extraccion de las variables
        f = Dataset(filename_list[fileidx])
        pres = getvar(f, "pressure", file_timeidx, method='cat')
        temp = getvar(f, var, file_timeidx, units= "m s-1", method='cat')
        temp2 = getvar(f, var2, file_timeidx, method='cat')
        
        #interpolacion al punto de presion
        for i,values in enumerate(lista):
            
            inter_var_temp = interplevel(temp,pres,values,meta=False)[a,b]
            var_temp[i,timeidx]  = inter_var_temp
            
            inter_var_temp2 = interplevel(temp2,pres,values,meta=False)[a,b]
            var_temp2[i,timeidx]  = inter_var_temp2
            
        f.close()
    #se guaradn los datos con el nombre de la variable
    np.save(station+fill[0][42:46]+'.npy',var_temp)
    np.save(station2+fill[0][42:46]+'.npy',var_temp2)

In [ ]:
# Se ejecuta de forma paralela la funcion
t1 = time.perf_counter()
#se debe sustituir para un iterador sobre fill la variable que contiene las direcciones.
for i in np.arange(2,11):
    fill = itemlist[365*(i-1):365*i]
    with concurrent.futures.ProcessPoolExecutor() as exe:
        exe.map(extrac_var_dynamics,var,station,var2,station2,grid_points[:,0],grid_points[:,1])
        
t2=time.perf_counter()
print("Tiempo total de exe...."+str(t2-t1))